<a href="https://colab.research.google.com/github/Bimawawoy/Belajar-V1/blob/main/Web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

###########IMPORT MODULE##########
#Main lib
import requests as re
from bs4 import BeautifulSoup
import time
import json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from time import sleep

#Decoration lib
import rich
from rich.panel import Panel as nel
from rich.console import Console
########################################

#Variable
cs = Console(width=50)
username = 'Bima'
index = 1
link = ['https://otakudesu.cloud', 'https://otakudesu.cloud/jadwal-rilis/', 'https://otakudesu.cloud/anime-list']

#try to ping web
try:
    response = re.get(link[0])
    ip = re.get('https://api.ipify.org/').text
    html = response.text
    statuscd = response.status_code
except Exception as e:
    cs.print(nel(f"[bold red]Terjadi Kesalahan : {e}"))
    statuscd = 404

#Fungsi Waktu
def get_waktu(jam):
    if 0 <= jam < 11:
        return "Pagi"
    elif 11 <= jam < 18:
        return "Sore"
    else:
        return "Malam"

def main():
    #Show status
    cs.rule("[bold cyan]Program Web Scraping CODE by BIMZ", style = "yellow")
    get_time = datetime.now().hour
    get_time_minute = datetime.now().minute
    get_time_rn = get_waktu(get_time)
    cs.print(nel.fit(f"[bold cyan on magenta]Selamat {get_time_rn},{username} :moyai:"))
    cs.print(nel.fit(f"[bold white on red]Link/Sumber : {link[0]}\n[bold white on yellow]Waktu saat ini : {get_time} : {get_time_minute}\n[bold white on green]Status code Web : {statuscd}\n[bold magenta]Ip kamu : {ip}\n[bold blue]Country : Unknown", border_style = "blue"))
    cs.rule("[bold cyan] Main Menu", style = "yellow")
    cs.print("[bold magenta]1.Trending Anime\n2.Jadwal rilis Anime\n3.List semua Anime A-Z[bold red ](Unstable)\n[bold magenta]4.Cari Anime[red](Beta)\n[bold magenta]5.Keluar", justify = "center")
    cs.print("[bold cyan]Created with :red_heart-emoji: by BIMZ", justify = "center")
    inp = cs.input("[bold red]$[BIMZ:smiley:] >>")
    if inp == "1":
       get_trending()
    elif inp == "2":
       release_date()
    elif inp == "3":
       list_anime()
    elif inp == "4":
        cari_anime()
    elif inp == "5":
        exit()
    else:
       cs.print("[bold red] Input tidak valid!")
       main()

#Fungsi Trending Anime
def get_trending():
    global index
    soup = BeautifulSoup(html, 'html.parser')

    # Judul
    titles = soup.find_all(class_='jdlflm')

    # Episode
    episodes = soup.find_all(class_='epz')

    # tanggal rilis
    release_dates = soup.find_all(class_='newnime')

    # Menampilkan data
    for title, episode, release_date in zip(titles, episodes, release_dates):
        cs.print(
        nel.fit(f"{index}.[bold green]{title.text.strip()}\n"
        f"[bold cyan]Episode: {episode.text.strip()}\n[bold magenta]Release Date: {release_date.text.strip()}", border_style = "red"))
        print('\n')
        index = index + 1
        time.sleep(0.3)
    index = index -1
    print(f"Total anime terkumpul : {index}")
    index = 1
    tanya()

#Fungsi Tanggal rilis
def release_date():
    response = re.get(link[1])
    soup = BeautifulSoup(response.text, 'html.parser')
    schedule_data = soup.find_all('div', class_='kglist321')

    # Inisialisasi list untuk menyimpan data
    days_list = []
    anime_count_list = []

    #Merekap data hari
    for day_data in schedule_data:
        day = day_data.find('h2').text
        animes = day_data.find_all('a')
        days_list.append(day)
        anime_count_list.append(len(animes))

        #Menampilkan data
        cs.rule(f'[cyan]{day}', style="red")
        for anime in animes:
            cs.print(anime.text)
            print('\n')
            time.sleep(0.3)

    #Membuat input menampilkan diagram
    datafr = cs.input("[bold cyan]Tampilkan diagram? (y/n)")
    if datafr.lower() == 'y':
        df = pd.DataFrame({'Day': days_list, 'Anime Count': anime_count_list})
        # Membuat diagram batang
        plt.bar(df['Day'], df['Anime Count'], color='cyan')
        plt.xlabel('Hari')
        plt.ylabel('Jumlah')
        plt.title('Jumlah rilis anime harian')
        with cs.status("Memuat diagram...", spinner = "monkey"):
           time.sleep(2)
           plt.show()
        tanya()
    else:
        main()


#Fungsi list Anime
def list_anime():

    #Memuat data
    with cs.status("Memuat data...(Mungkin agak lama karena datanya banyak :v)", spinner="monkey"):
        time.sleep(3)
    response = re.get(link[2])
    soup = BeautifulSoup(response.text, "html.parser")

    #Mencari class
    sections = soup.find_all(class_="jdlbar")
    anime_dict = {}

    #Merekap anime dan menyimpan dalam variable
    for section in sections:
        title = section.find("ul").find("li").text.strip()
        category = title[0]
        anime_list = [anime.text.strip() for anime in section.find_all("a")]
        anime_dict.setdefault(category, []).extend(anime_list)

    # Menampilkan list Anime dengan perulangan
    for category, anime_list in sorted(anime_dict.items()):
        cs.rule(f"[bold red]{category}", style="yellow")
        cs.print(f"• " + '\n• '.join(sorted(set(anime_list))) + "\n")
        time.sleep(0.3)

    #Mengambil input
    cts = cs.input("[bold cyan]Tampilkan diagram? (y/n) :")
    if cts.lower() == 'y':
       categories = sorted(anime_dict.keys())
       anime_counts = [len(anime_list) for anime_list in anime_dict.values()]
       plt.figure(figsize=(10, 6))
       plt.bar(categories, anime_counts, color='skyblue')
       plt.xlabel('Kategori')
       plt.ylabel('Jumlah Anime')
       plt.title('Jumlah Anime per Kategori')
       with cs.status("Memuat diagram..."):
           time.sleep(1)
       plt.show()
       tanya()
    else:
        main()

#Fungsi Cari Anime
def cari_anime():

    #Membuat input untuk cari Anime
    cari = cs.input("[bold red]Cari Anime : :warning:(type 'exit' to exit)")
    if cari == '':
       cs.print("Tidak boleh kosong!")
       cari_anime()
    elif cari.lower() == 'exit':
        main()

    #Mengganti link sesuai input
    linkcari = f"https://otakudesu.cloud/?s={cari}&post_type=anime"

    #Mendapatkan elemen html
    response = re.get(linkcari)
    soup = BeautifulSoup(response.text, "html.parser")

    #Merekap semua element html yg memiliki tag'ul'
    anime_list = soup.find_all("ul", class_="chivsrc")

    #Mencoba mendapatkan data dari html(scrape)
    try:
       for anime in anime_list:
           title = anime.h2.text.strip()
           rating = 'gagal mendapatkan data rating'
           status = anime.find("div", class_="set").find_next("div", class_="set").text.split(":")[1].strip()
           genres = [genre.text.strip() for genre in anime.find_all("a", rel="tag")]
           linkk = anime.h2.a.get("href")
           get_sinopsis = re.get(linkk)
           soup_linkk = BeautifulSoup(get_sinopsis.text, 'html.parser')
           sinopsis = soup_linkk.find('div', class_='sinopc').text
           if sinopsis == '':
              sinopsis = 'Sinopsis tidak tersedia!(Null)'
           skor_element = soup_linkk.select_one('p:contains("Skor:") span')
           skor = skor_element.text.strip() if skor_element else "Not Found"

           # Mengambil tanggal rilis
           tanggal_rilis_element = soup_linkk.select_one('p:contains("Tanggal Rilis:") span')
           tanggal_rilis = tanggal_rilis_element.text.strip() if tanggal_rilis_element else "Not Found"

           # Mengambil produser
           produser_element = soup_linkk.select_one('p:contains("Produser:") span')
           produser = produser_element.text.strip() if produser_element else "Not Found"


           #Menampilkan Hasil
           print("\n")#Ini margin/memberi jarak
           cs.print(":thumbs_up: Anime ditemukan!")
           cs.print(nel.fit(f"[bold green]Judul : {title}\n[magenta]Status : {status}\n[white]{skor}\n[bold yellow]{tanggal_rilis}\n[bold cyan]{produser}\n[red]Genre : {', '.join(genres)}\n[bold blue]Sinopsis : {sinopsis} ", border_style = "cyan"))
           print(f"Link: {linkk}")
           print("-" * 20)

    #Mengontrol error jika anime tidak ditemukan
    except AttributeError:
           cs.print(":thumbs_down: Anime tidak ditemukan!")
    cari_anime()



def tanya():

     #Fungsi kembali ke menu utama/awal
    iqn = cs.input("[bold cyan] Back to menu? (y/n) : ")
    if iqn.lower() == 'y':
       main()
    else:
       exit()




#First time setup
if __name__ == "__main__" :
    with cs.status("Memuat data...", spinner = "monkey"):
       time.sleep(2)
    main()

🙉  Memuat data...

─────── Program Web Scraping CODE by BIMZ ────────

╭──────────────────────╮
│ Selamat Pagi,Bima 🗿 │
╰──────────────────────╯

╭───────────────────────────────────────╮
│ Link/Sumber : https://otakudesu.cloud │
│ Waktu saat ini : 8 : 22               │
│ Status code Web : 200                 │
│ Ip kamu : 34.125.40.173               │
│ Country : Unknown                     │
╰───────────────────────────────────────╯

───────────────────  Main Menu ───────────────────

...

👍 Anime ditemukan!

╭────────────────────────────────────────────────╮
│ Judul : Bocchi the Rock! (Episode 1 – 12)      │
│ Subtitle Indonesia                             │
│ Status : Completed                             │
│ Skor: 9.00                                     │
│ Tanggal Rilis: Okt 09, 2022                    │
│ Produser: Aniplex, Houbunsha                   │
│ Genre : CGDCT, Comedy, Music, Slice of Life,   │
│ Comedy, School, Shounen, Slice of Life         │
│ Sinopsis : Cerita berpusat ke Gotou Hitori,    │
│ seorang gadis SMA yang baru belajar bermain    │
│ gitar. Ia memilih bermain Gitar dengan tujuan  │
│ membentuk sebuah band. Sayangnya, tujuan ini   │
│ tidak berhasil terwujud sampai ia bertemu      │
│ dengan Ijichi Nijika, seorang pemain           │
│ drum.Menariknya, Nijika rupanya tengah mencari │
│ seseorang yang bisa bermain gitar. Untuk itu,  │
│ Nijika mengajak Gotou agar bergabung dengan    │
│ formasi band miliknya. Melihat kesempatan yang │
│ langka ini, Gotou tanpa pikir panjang menerima │
│ tawaran Nijika dan membentuk band bernama      │
│ Kessoku.                                       │
╰────────────────────────────────────────────────╯

Link: https://otakudesu.cloud/anime/bochi-rock-sub-indo/
--------------------


Cari Anime : ⚠(type 'exit' to exit)

KeyboardInterrupt: Interrupted by user